In [0]:
%fs mkdirs dbfs:/FileStore/msba-trends/raw-from-kafka

res1: Boolean = true

In [0]:
"""
        SELECT key, timestamp,   value.ID,
            value.Source                 ,
            value.Severity               ,
            value.Start_Time             ,
            value.End_Time               ,
            value.Start_Lat              ,
            value.Start_Lng              ,
            value.End_Lat                ,
            value.End_Lng                ,
            value.Distance_miles         ,
            value.Description            ,
            value.Street                 ,
            value.City                   ,
            value.County                 ,
            value.State                  ,
            value.Zipcode                ,
            value.Country                ,
            value.Timezone               ,
            value.Airport_Code           ,
            value.Weather_Timestamp      ,
            value.Temperature_F          ,
            value.Wind_Chill_F           ,
            value.Humidity_percent       ,
            value.Pressure_in            ,
            value.Visibility_mi          ,
            value.Wind_Direction         ,
            value.Wind_Speed_mph         ,
            value.Precipitation_in       ,
            value.Weather_Condition      ,
            value.Amenity                ,
            value.Bump                   ,
            value.Crossing               ,
            value.Give_Way               ,
            value.Junction               ,
            value.No_Exit                ,
            value.Railway                ,
            value.Roundabout             ,
            value.Station                ,
            value.Stop                   ,
            value.Traffic_Calming        ,
            value.Traffic_Signal         ,
            value.Turning_Loop           , 
            value.Sunrise_Sunset         ,
            value.Civil_Twilight         ,
            value.Nautical_Twilight      ,
            value.Astronomical_Twilight
""".replace(" ", "").replace("\n", "").replace(",", "', '")

"SELECTkey', 'timestamp', 'value.ID', 'value.Source', 'value.Severity', 'value.Start_Time', 'value.End_Time', 'value.Start_Lat', 'value.Start_Lng', 'value.End_Lat', 'value.End_Lng', 'value.Distance_miles', 'value.Description', 'value.Street', 'value.City', 'value.County', 'value.State', 'value.Zipcode', 'value.Country', 'value.Timezone', 'value.Airport_Code', 'value.Weather_Timestamp', 'value.Temperature_F', 'value.Wind_Chill_F', 'value.Humidity_percent', 'value.Pressure_in', 'value.Visibility_mi', 'value.Wind_Direction', 'value.Wind_Speed_mph', 'value.Precipitation_in', 'value.Weather_Condition', 'value.Amenity', 'value.Bump', 'value.Crossing', 'value.Give_Way', 'value.Junction', 'value.No_Exit', 'value.Railway', 'value.Roundabout', 'value.Station', 'value.Stop', 'value.Traffic_Calming', 'value.Traffic_Signal', 'value.Turning_Loop', 'value.Sunrise_Sunset', 'value.Civil_Twilight', 'value.Nautical_Twilight', 'value.Astronomical_Twilight"

In [0]:
from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

In [0]:
class Consumer:
    def __init__(self):
        self.base_path = "/Volumes/msba_6331_group1/default/csom-msba6311-g1/"
        self.ckp_path = f"{self.base_path}/checkpoint/kafka_consumer"
        # self.src_data_folder = f"{self.base_path}/producer_json/start_yr_mnth=2016-03/*.json"
        self.BOOTSTRAP_SERVER = "pkc-4r087.us-west2.gcp.confluent.cloud:9092"
        self.security_protocol="SASL_SSL"
        self.sasl_mechanisms="PLAIN"
        # self.client_id= "avsk_local"
        # Required connection configs for Kafka producer, consumer, and admin
        self.CLUSTER_API_KEY = ""
        self.CLUSTER_API_SECRET = ""
        self.JAAS_MODULE = "org.apache.kafka.common.security.plain.PlainLoginModule"
        self.topic_name = "us_acc_test"

    def get_schema(self):
        schema = """ 
        ID                     string,
        Source                 string,
        Severity               int,
        Start_Time             string,
        End_Time               string,
        Start_Lat              float,
        Start_Lng              float,
        End_Lat                float,
        End_Lng                float,
        Distance_miles           float,
        Description            string,
        Street                 string,
        City                   string,
        County                 string,
        State                  string,
        Zipcode                string,
        Country                string,
        Timezone               string,
        Airport_Code           string,
        Weather_Timestamp      string,
        Temperature_F         float,
        Wind_Chill_F          float,
        Humidity_percent            float,
        Pressure_in           float,
        Visibility_mi         float,
        Wind_Direction         string,
        Wind_Speed_mph        float,
        Precipitation_in      float,
        Weather_Condition      string,
        Amenity                string,
        Bump                   string,
        Crossing               string,
        Give_Way               string,
        Junction               string,
        No_Exit                string,
        Railway                string,
        Roundabout             string,
        Station                string,
        Stop                   string,
        Traffic_Calming        string,
        Traffic_Signal         string,
        Turning_Loop           string,  
        Sunrise_Sunset         string, 
        Civil_Twilight         string,
        Nautical_Twilight      string,
        Astronomical_Twilight  string
        """
        return schema
    
    def get_inner_kafka_columns(self):
        select_string = """
        SELECT key, timestamp,   value.ID,
            value.Source                 ,
            value.Severity               ,
            value.Start_Time             ,
            value.End_Time               ,
            value.Start_Lat              ,
            value.Start_Lng              ,
            value.End_Lat                ,
            value.End_Lng                ,
            value.Distance_miles         ,
            value.Description            ,
            value.Street                 ,
            value.City                   ,
            value.County                 ,
            value.State                  ,
            value.Zipcode                ,
            value.Country                ,
            value.Timezone               ,
            value.Airport_Code           ,
            value.Weather_Timestamp      ,
            value.Temperature_F          ,
            value.Wind_Chill_F           ,
            value.Humidity_percent       ,
            value.Pressure_in            ,
            value.Visibility_mi          ,
            value.Wind_Direction         ,
            value.Wind_Speed_mph         ,
            value.Precipitation_in       ,
            value.Weather_Condition      ,
            value.Amenity                ,
            value.Bump                   ,
            value.Crossing               ,
            value.Give_Way               ,
            value.Junction               ,
            value.No_Exit                ,
            value.Railway                ,
            value.Roundabout             ,
            value.Station                ,
            value.Stop                   ,
            value.Traffic_Calming        ,
            value.Traffic_Signal         ,
            value.Turning_Loop           , 
            value.Sunrise_Sunset         ,
            value.Civil_Twilight         ,
            value.Nautical_Twilight      ,
            value.Astronomical_Twilight
        FROM accident_new_batch
        """
        df = spark.sql(select_string)
        return df

    # def preprocess(self):
    #     #flatten value column
    #     df = self.get_inner_kafka_columns()

    #     #format columns
    #     df = df.withColumn("Start_Time", F.to_timestamp(df.Start_Time)) \
    #     .withColumn("End_Time", F.to_timestamp(df.End_Time)) \
    #     .withColumn("Severity", df.Severity.cast("double")) \
    #     .withColumn("accident_duration_mins", ((F.col("End_Time").cast("long") - F.col("Start_Time").cast("long"))/60))

    #     pipeline_path = "dbfs:/FileStore/artifacts/pipeline_accidents/logistic"
    #     pipeline_model_path = "dbfs:/FileStore/artifacts/pipeline_model_accidents/logistic"

    #     pipeline_for_inference = Pipeline.load(pipeline_path)
    #     pipeline_model_for_inference = PipelineModel.load(pipeline_model_path)
    #     df = pipeline_model_for_inference.transform(df)
    #     return df.selectExpr('timestamp','ID','Source', 'prediction', 'Severity','Start_Time','End_Time','Start_Lat','Start_Lng','End_Lat','End_Lng','Distance_miles','Description','Street','City','County','State','Zipcode','Country','Timezone','Airport_Code','Weather_Timestamp','Temperature_F','Wind_Chill_F','Humidity_percent','Pressure_in','Visibility_mi','Wind_Direction','Wind_Speed_mph','Precipitation_in','Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight')
    
    def stream_consumer(self, startTime = 1):
        kafka_df = ( spark.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers", self.BOOTSTRAP_SERVER)
            .option("kafka.security.protocol", self.security_protocol)
            .option("kafka.sasl.mechanism", self.sasl_mechanisms)
            .option("kafka.sasl.jaas.config", f"{self.JAAS_MODULE} required username='{self.CLUSTER_API_KEY}' password='{self.CLUSTER_API_SECRET}';")
            .option("subscribe", self.topic_name)
            .option("maxOffsetsPerTrigger", 10) #ensures each micro-batch doesn't run too long by dividing records uniformly
            # .option("startingTimestamp", startTime)
            .option("startingOffsets", "earliest")
            .load()
        )

        # kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()
        return kafka_df
    
    def getAccidentdata(self, kafka_df):
        from pyspark.sql.functions import from_json

        kafka_df2 = kafka_df.select(kafka_df.key.cast("string").alias("key"),
                        from_json(kafka_df.value.cast("string"), schema=self.get_schema()).alias("value"),
                        "topic", "timestamp")
        return kafka_df2.selectExpr('timestamp', 'value.ID as ID', 'value.Source as Source', 'value.Severity as Severity', 'value.Start_Time as Start_Time', 'value.End_Time as End_Time', 'value.Start_Lat as Start_Lat', 'value.Start_Lng as Start_Lng', 'value.End_Lat as End_Lat', 'value.End_Lng as End_Lng', 'value.Distance_miles as Distance_miles', 'value.Description as Description', 'value.Street as Street', 'value.City as City', 'value.County as County', 'value.State as State', 'value.Zipcode as Zipcode', 'value.Country as Country', 'value.Timezone as Timezone', 'value.Airport_Code as Airport_Code', 'value.Weather_Timestamp as Weather_Timestamp', 'value.Temperature_F as Temperature_F', 'value.Wind_Chill_F as Wind_Chill_F', 'value.Humidity_percent as Humidity_percent', 'value.Pressure_in as Pressure_in', 'value.Visibility_mi as Visibility_mi', 'value.Wind_Direction as Wind_Direction', 'value.Wind_Speed_mph as Wind_Speed_mph', 'value.Precipitation_in as Precipitation_in', 'value.Weather_Condition as Weather_Condition', 'value.Amenity as Amenity', 'value.Bump as Bump', 'value.Crossing as Crossing', 'value.Give_Way as Give_Way', 'value.Junction as Junction', 'value.No_Exit as No_Exit', 'value.Railway as Railway', 'value.Roundabout as Roundabout', 'value.Station as Station', 'value.Stop as Stop', 'value.Traffic_Calming as Traffic_Calming', 'value.Traffic_Signal as Traffic_Signal', 'value.Turning_Loop as Turning_Loop', 'value.Sunrise_Sunset as Sunrise_Sunset', 'value.Civil_Twilight as Civil_Twilight', 'value.Nautical_Twilight as Nautical_Twilight', 'value.Astronomical_Twilight as Astronomical_Twilight')
        
        
    # implement idompotence using upsert
    def upsert(self, accidentDF, batch_id):
        """
        The merge is happening between a new batch and the delta table.
        """
        accidentDF = accidentDF.withColumn("Start_Time", F.to_timestamp(accidentDF.Start_Time)) \
        .withColumn("End_Time", F.to_timestamp(accidentDF.End_Time)) \
        .withColumn("Severity", accidentDF.Severity.cast("double")) \
        .withColumn("accident_duration_mins", ((F.col("End_Time").cast("long") - F.col("Start_Time").cast("long"))/60)) 
        # \.createOrReplaceTempView("acc_df_temp_view")

        pipeline_path = "dbfs:/FileStore/artifacts/pipeline_accidents/logistic"
        pipeline_model_path = "dbfs:/FileStore/artifacts/pipeline_model_accidents/logistic"

        pipeline_for_inference = Pipeline.load(pipeline_path)
        pipeline_model_for_inference = PipelineModel.load(pipeline_model_path)
        df = pipeline_model_for_inference.transform(accidentDF)
        df = df.selectExpr('timestamp','ID','Source', 'prediction', 'Severity','Start_Time','End_Time','Start_Lat','Start_Lng','End_Lat','End_Lng','Distance_miles','Description','Street','City','County','State','Zipcode','Country','Timezone','Airport_Code','Weather_Timestamp','Temperature_F','Wind_Chill_F','Humidity_percent','Pressure_in','Visibility_mi','Wind_Direction','Wind_Speed_mph','Precipitation_in','Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight')

        df.createOrReplaceTempView("acc_df_temp_view")
        # accidentDF.registerTempTable('accident_new_batch')
        # print(accidentDF.count())
        # predicted_df = self.preprocess()
        # predicted_df.createOrReplaceTempView('predicted_df')

        merge_statement = """ MERGE INTO msba_6331_group1.default.accidents_kafka_ingest delta 
        USING acc_df_temp_view raw 
        ON
        delta.id == raw.id
        WHEN MATCHED THEN
        UPDATE SET *
        WHEN NOT MATCHED THEN
        INSERT *
        """
        accidentDF._jdf.sparkSession().sql(merge_statement) # get same session to execute sql
        print("done merge!!!!!!!!!!!!")

    def process(self, startingTime = 1):
        """
        Need to add model prediction column at last in the delta table to store real-time prediction 
        """
        print(f"Starting Consumer Stream...", end=' ')
        rawDF = self.stream_consumer(startingTime)
        accidentDF = self.getAccidentdata(rawDF)
        # print(accidentDF.columns)
        sQuery =  ( accidentDF.writeStream
                        .queryName("kafka-read-idompotent")
                        .foreachBatch(self.upsert) #foreachBatch can be used to write into different sinks by applying custom logic. The acc_DF dataframe is passed as a fn argument.
                        .option("checkpointLocation", f"{self.ckp_path}")
                        .outputMode("append")
                    #     .toTable("accidents_bz")     we don't want to write to a table instead do upsert       
                        .start() #adding an action start() since we commented toTable()
                )
        print("Done")
        return sQuery

In [0]:
# dbutils.fs.rm("dbfs:/FileStore/msba_project_data/kafka_to_json/", recurse=True)

In [0]:
c = Consumer()
sQuery = c.process()

Starting Consumer Stream... Done


In [0]:
sQuery.stop()

In [0]:
%sql
USE CATALOG msba_6331_group1;
USE SCHEMA default;

In [0]:
import time
while True:
  print(spark.sql("select count(*) from accidents_kafka_ingest").collect()[0][0])
  # print(spark.sql("select concat(prediction--severity) from accidents_kafka_ingest").collect()[0][0])
  spark.sql("REFRESH TABLE accidents_kafka_ingest")
  time.sleep(2)

148
148
148
148
148
done merge!!!!!!!!!!!!
156
156
156
156
156
done merge!!!!!!!!!!!!
162
162
162
162
done merge!!!!!!!!!!!!
168
168
168
168
168
done merge!!!!!!!!!!!!
174
174
174
174
done merge!!!!!!!!!!!!
180
180
180
180
180
done merge!!!!!!!!!!!!
186
186
186
186
done merge!!!!!!!!!!!!
192
192
192
192
192
done merge!!!!!!!!!!!!
199
199
199
199
199
done merge!!!!!!!!!!!!
205
205
205
205
205
done merge!!!!!!!!!!!!
211
211
211
211
211
done merge!!!!!!!!!!!!
217
217
217
217
217
done merge!!!!!!!!!!!!
225
225
225
225
done merge!!!!!!!!!!!!
231
231
231
231
231
done merge!!!!!!!!!!!!
236
236
236
236
236
done merge!!!!!!!!!!!!
245
245
245
245
done merge!!!!!!!!!!!!
251
251
251
251
251
done merge!!!!!!!!!!!!
261
261
261
261
261
done merge!!!!!!!!!!!!
271
271
271
271
done merge!!!!!!!!!!!!
280
280
280
280
280
done merge!!!!!!!!!!!!
290
290
290
290
290
done merge!!!!!!!!!!!!
300
300
300
300
300
done merge!!!!!!!!!!!!
310
310
310
310
310
310
310
310
310
310
310
310
310
310
310
310
310
310
310
31

In [0]:
spark.sql("SELECT COUNT(*) FROM accidents_kafka_ingest").createOrReplaceTempView("tmp")

In [0]:
spark.sql("select * from accidents_kafka_ingest").display()

timestamp,ID,Source,prediction,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance_miles,Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature_F,Wind_Chill_F,Humidity_percent,Pressure_in,Visibility_mi,Wind_Direction,Wind_Speed_mph,Precipitation_in,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
2023-12-05 11:37:59.333,A-4603124,Source1,0,0.0,2023-01-27 14:29:50,2023-01-27 16:41:56,33.902767,-79.2338,33.898792,-79.23711,0.334,Incident on JORDANVILLE RD near PEE DEE RD Drive with caution.,Jordanville Rd,Galivants Ferry,Horry,SC,29544,US,US/Eastern,KMAO,2023-01-27 14:35:00,50.0,50.0,32.0,30.12,10.0,SW,8.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
2023-12-05 11:37:59.331,A-5224328,Source1,0,0.0,2023-02-10 02:22:00,2023-02-10 04:53:17,33.854248,-117.93255,33.854244,-117.93769,0.295,Accident on Riverside Fwy W - CA-91 W from Harbor Blvd (CA-91) to Euclid St (CA-91).,CA-91 W,Fullerton,Orange,CA,92832,US,US/Pacific,KFUL,2023-02-10 01:53:00,63.0,63.0,17.0,30.04,10.0,NNW,3.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
2023-12-05 11:37:59.24,A-3939683,Source1,0,0.0,2023-02-07 06:06:30,2023-02-07 09:37:00,34.078472,-118.2286,34.052483,-118.25933,2.514,Stationary traffic on Arroyo Seco Pkwy - CA-110 S - Pasadena Fwy S - Harbor Fwy S from I-5 (Los Angeles) (West) (CA-110) to 5th St/6th St (CA-110) due to accident.,Arroyo Seco Pkwy,Los Angeles,Los Angeles,CA,90012,US,US/Pacific,KCQT,2023-02-07 05:54:00,46.0,46.0,61.0,30.04,4.0,CALM,0.0,0.0,Haze,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Day,Day
2023-12-05 11:37:59.34,A-4011112,Source1,0,0.0,2023-02-02 09:42:00,2023-02-02 10:07:30,34.279175,-109.8004,34.281673,-109.80878,0.508,Stationary traffic on US-60 W - US-60 E from CR-3140/Vernon McNary Rd (US-60) to AZ-61 (US-60) due to accident.,US Highway 60,Show Low,Apache,AZ,85901,US,US/Mountain,KSOW,2023-02-02 09:35:00,28.0,22.0,59.0,23.99,10.0,NNW,6.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
2023-12-05 11:37:59.237,A-4162469,Source1,0,0.0,2023-01-24 12:27:47,2023-01-24 14:37:48,45.240383,-121.70002,45.23177,-121.70221,0.605,A disabled vehicle is creating a hazard causing minimal delay to traffic. Use caution.,E Highway 26,Government Camp,Clackamas,OR,97028,US,US/Pacific,KCZK,2023-01-24 12:53:00,50.0,50.0,61.0,30.56,10.0,CALM,0.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
2023-12-05 11:37:59.331,A-4133159,Source1,0,0.0,2023-01-13 11:18:59,2023-01-13 12:34:04,41.449883,-81.544,41.449875,-81.540825,0.164,Incident on HARVARD near HARVARD Drive with caution.,Harvard Ave,Beachwood,Cuyahoga,OH,44122-6812,US,US/Eastern,KCGF,2023-01-13 11:45:00,30.0,20.0,93.0,29.01,2.0,N,13.0,0.0,Light Snow,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
2023-12-05 11:37:59.237,A-5096647,Source1,0,0.0,2023-01-25 12:59:05,2023-01-25 14:16:13,29.965887,-90.1097,29.96624,-90.10929,0.035,Incident on S CARROLLTON AVE near HOUSE 3801 Drive with caution.,Washington Ave,New Orleans,Orleans,LA,70125-1050,US,US/Pacific,KAUD,2023-01-25 12:55:00,54.0,54.0,59.0,29.99,10.0,NNW,13.0,0.0,Partly Cloudy,False,False,True,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day
2023-12-05 11:37:59.334,A-4803860,Source1,0,0.0,2023-01-26 12:29:42,2023-01-26 13:46:25,29.927622,-90.02683,29.928122,-90.02801,0.079,Incident on GENERAL DE GAULLE DR near HOUSE 2916 Drive with caution.,General Degaulle Dr,New Orleans,Orleans,LA,70114-6440,US,US/Pacific,KAUD,2023-01-26 12:55:00,56.0,56.0,42.0,30.29,10.0,NW,13.0,0.0

In [0]:
%sql
SELECT * FROM accidents_kafka_ingest limit 10;

timestamp,ID,Source,prediction,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance_miles,Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature_F,Wind_Chill_F,Humidity_percent,Pressure_in,Visibility_mi,Wind_Direction,Wind_Speed_mph,Precipitation_in,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight


In [0]:
def insert_to_final_table():
    """
    Need to add model prediction column at last
    """
    insert_string = """
    INSERT OVERWRITE TABLE msba_6331_group1.default.accidents_final
    SELECT value.ID,
        value.Source                 ,
        value.Severity               ,
        value.Start_Time             ,
        value.End_Time               ,
        value.Start_Lat              ,
        value.Start_Lng              ,
        value.End_Lat                ,
        value.End_Lng                ,
        value.Distance_miles         ,
        value.Description            ,
        value.Street                 ,
        value.City                   ,
        value.County                 ,
        value.State                  ,
        value.Zipcode                ,
        value.Country                ,
        value.Timezone               ,
        value.Airport_Code           ,
        value.Weather_Timestamp      ,
        value.Temperature_F          ,
        value.Wind_Chill_F           ,
        value.Humidity_percent       ,
        value.Pressure_in            ,
        value.Visibility_mi          ,
        value.Wind_Direction         ,
        value.Wind_Speed_mph         ,
        value.Precipitation_in       ,
        value.Weather_Condition      ,
        value.Amenity                ,
        value.Bump                   ,
        value.Crossing               ,
        value.Give_Way               ,
        value.Junction               ,
        value.No_Exit                ,
        value.Railway                ,
        value.Roundabout             ,
        value.Station                ,
        value.Stop                   ,
        value.Traffic_Calming        ,
        value.Traffic_Signal         ,
        value.Turning_Loop           , 
        value.Sunrise_Sunset         ,
        value.Civil_Twilight         ,
        value.Nautical_Twilight      ,
        value.Astronomical_Twilight
    FROM accidents_kafka_ingest
    """
    spark.sql(insert_string)
    print("inserted into final table!!!!!!!")


In [0]:
insert_to_final_table()

inserted into final table!!!!!!!


In [0]:
%sql
SELECT *
 FROM accidents_final limit 10;

ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance_miles,Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature_F,Wind_Chill_F,Humidity_percent,Pressure_in,Visibility_mi,Wind_Direction,Wind_Speed_mph,Precipitation_in,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
A-165541,Source2,2,2016-03-23 18:21:02,2016-03-23 19:21:02,41.25887,-96.09101,null,null,0.0,Accident on 114th St at Burke St.,N 114th St,Omaha,Douglas,NE,68154-3162,US,US/Central,KMLE,2016-03-23 18:15:00,41.0,33.1,88.0,29.52,10.0,NE,15.0,0.01,Light Rain,false,false,true,false,false,false,false,false,false,true,false,false,false,Day,Day,Day,Day
A-3413312,Source1,2,2016-03-04 05:41:45,2016-03-04 11:41:45,41.06076,-81.45923,41.06341,-81.4506,0.485,At OH-18/Market St/Exit 26 - Accident.,I-76 E,Akron,Summit,OH,44312,US,US/Eastern,KAKR,2016-03-04 05:42:00,28.9,25.2,89.0,30.01,2.5,NE,3.5,0.0,Light Snow,false,false,false,false,false,false,false,false,false,false,false,false,false,Night,Night,Night,Day
A-3516580,Source1,2,2016-03-28 09:40:38,2016-03-28 15:40:38,40.02645,-75.23124,40.02527,-75.22897,0.145,At Belmont Ave/Exit 338 - Accident.,I-76 E,Bala Cynwyd,Montgomery,PA,19004,US,US/Eastern,KLOM,2016-03-28 09:35:00,44.6,43.1,100.0,29.68,5.0,ENE,3.5,0.02,null,false,false,false,false,false,false,false,false,false,false,false,false,false,Day,Day,Day,Day
A-3418463,Source1,2,2016-03-29 20:36:43,2016-03-30 02:36:43,34.583164,-118.34332,34.583294,-118.34178,0.088,At CR-N2/Elizabeth Lake Rd - Accident.,Bouquet Canyon Rd,Santa Clarita,Los Angeles,CA,91390-1164,US,US/Pacific,K3A6,2016-03-29 18:51:00,56.0,56.0,51.0,29.01,10.0,ESE,12.0,0.0,Mostly Cloudy,false,false,false,false,false,false,false,false,false,false,false,false,false,Night,Night,Night,Day
A-3413245,Source1,4,2016-03-02 12:04:52,2016-03-02 18:04:52,39.43794,-82.53893,39.44135,-82.56107,1.205,Closed between Culp Rd and Unger Rd - Road closed due to accident.,State Route 374,Logan,Hocking,OH,43138-9792,US,US/Eastern,KLHQ,2016-03-02 11:53:00,27.0,18.7,66.0,30.19,9.0,West,8.1,0.0,Light Snow,false,false,false,false,false,false,false,false,false,false,false,false,false,Day,Day,Day,Day
A-3413247,Source1,3,2016-03-02 05:10:40,2016-03-02 19:10:40,38.07759,-84.45712,38.10084,-84.48764,2.31,Between US-27/US-68/Exit 113 and KY-922/Exit 115 - Accident.,I-75 N,Lexington,Fayette,KY,40505,US,US/Eastern,KLEX,2016-03-02 04:54:00,28.9,21.1,69.0,30.11,9.0,NW,8.1,0.0,Overcast,false,false,false,false,false,false,false,false,false,false,false,false,false,Night,Night,Night,Night
A-97623,Source2,2,2016-03-24 19:23:57,2016-03-24 19:53:57,34.36511,-118.503716,null,null,0.01,Accident on CA-14 Southbound at Exit 2 Newhall Ave.,Newhall Ave,Newhall,Los Angeles,CA,91321,US,US/Pacific,K3A6,2016-03-24 19:51:00,71.0,71.0,25.0,29.09,10.0,N,15.0,0.0,Fair,false,false,false,false,false,false,false,false,false,false,false,false,false,Night,Day,Day,Day
A-165540,Source2,2,2016-03-23 18:08:20,2016-03-23 19:08:20,41.219727,-96.043106,null,null,0.01,Serious accident on F St at 84th St.,F St,Omaha,Douglas,NE,68127-1735,US,US/Central,KMLE,2016-03-23 18:15:00,41.0,33.1,88.0,29.52,10.0,NE,15.0,0.01,Light Rain,false,false,false,false,false,false,false,false,false,false,false,true,false,Day,Day,Day,Day
A-98734,Source2,3,2016-03-30 18:22:38,2016-03-30 18:52:38,33.92471,-116.91184,null,null,0.0,Accident on I-10 Westbound at Exit 98 Sunset Ave.,I-10 W,Banning,Riverside,CA,92220,US,US/Pacific,KBUO,2016-03-30 18:15:00,57.0,57.0,38.0,28.32,10.0,NNE,7.0,0.0,Thunder in the Vicinity,false,false,false,false,false,false,false,false,false,false,false,false,false,Day,Day,Day,Day
